In [48]:
import os

import shutil

import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F

import torchvision
from torchvision import models, datasets, transforms
from torchvision.datasets import ImageFolder

from PIL import Image
import matplotlib.pyplot as plt

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {DEVICE} for inference')

Using cuda for inference


#### Model Settings #####

In [49]:
TRAIN_SPLIT = 0.7
TEST_SPLIT = 0.3

LEARNING_RATE = 0.001

BATCH_SIZE = 1
NUM_EPOCHS = 10

NUM_CLASSES = 2 # cat or dog

##### Sort Dataset Into Folders ####

In [50]:
# I will split the training dataset into training and test (test = 30% of training data)

base_dir = "Cat Vs Dog Dataset"
train_dir = os.path.join(base_dir,"train")

if not os.path.exists(f"{train_dir}/cat"):
        os.makedirs(f"{train_dir}/cat")
        
if not os.path.exists(f"{train_dir}/dog"):
        os.makedirs(f"{train_dir}/dog")

# Sort cats and dog in Train dir into respective folders
onlyFilesinTrain = [filename for filename in os.listdir(train_dir) if not os.path.isdir(os.path.join(train_dir,filename))]

for filename in onlyFilesinTrain:
        
        label = filename.split(".")[0]
        
        if(label == "cat"):
                shutil.move(f"{train_dir}/{filename}" , f"{train_dir}/cat/{filename}")
                
        elif(label == "dog"):
                shutil.move(f"{train_dir}/{filename}" , f"{train_dir}/dog/{filename}")
        
        else:
                print("NO Label")



#### Load Data #####

In [51]:
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

dataset = datasets.ImageFolder(train_dir)

DATASET_SIZE = len(dataset)

train_data,test_data = random_split(dataset, [ int(TRAIN_SPLIT*DATASET_SIZE) , int(TEST_SPLIT*DATASET_SIZE)])

train_loader = DataLoader(dataset=train_data, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_data, 
                         batch_size=BATCH_SIZE, 
                         shuffle=False)

CLASSES = dataset.class_to_idx #dataset.classes  

#### Load Model ####

In [52]:
resnet18 = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
print(resnet18.fc)

# we only need 2 classes need to cahnge ouput layer
resnet18.fc = nn.Linear(resnet18.fc.in_features,NUM_CLASSES)
print()
print(resnet18.fc)

# Make sure to send our model to GPU if available
resnet18 = resnet18.to(DEVICE)

Linear(in_features=512, out_features=1000, bias=True)

Linear(in_features=512, out_features=2, bias=True)
